### 引入相關python模組

In [ ]:
from mycnn import UNet
from mycnn import data
from mycnn import utils
import tensorflow as tf
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt

### 自動建立VOC分割資料集

會自動在工作路徑底下建立資料夾，並建立相關的資料集檔案結構  
也會檢查路徑底下是否已經有建立完成檔案，避免重複下載及建立

In [ ]:
data.download_pascal_voc_dataset(
    "./datasets"
)
data.make_voc_segment_dataset(
    "./datasets/VOC/VOCdevkit/VOC2012",
    "./datasets"
)

### 使用 `tf.data.Dataset` 來載入分割資料集

使用自建的 Dataset 實例來讀取影像、遮罩(mask)，並處理成 CNN 可以訓練的形式

```
mycnn.data.generate_segmentation_dataset

參數名稱            型態    說明
directory        : str   : 資料路徑 (子資料夾為類別)
image_size       : tuple : 影像大小
batch_size       : int   : 批次大小
subtract_mean    : float : 減去影像的均值，使其正規化
divide_stddev    : float : 除去影像標準差，使其正規化
shuffle_filepath : bool  : 打亂資料檔案路徑順序
shuffle_dataset  : bool  : 打亂資料讀取順序
validation_split : float : 分離驗證集的比例
```

In [ ]:
train_dataset, valid_dataset = data.generate_segmentation_dataset(
    "./datasets/VOCSegmentation/train",
    image_size=(572,572),
    mask_size=(388,388),
    batch_size=1,
    subtract_mean=0,
    divide_stddev=255,
    shuffle_filepath=True,
    # shuffle_dataset=True,
    validation_split=0.1,
    mask_mode=1,  # for original U-net
)
train_image_paths = train_dataset.image_paths
valid_image_paths = valid_dataset.image_paths
train_mask_paths = train_dataset.mask_paths
valid_mask_paths = valid_dataset.mask_paths

### 檢查原始資料

In [ ]:
for i, m in train_dataset.take(1):
    bi = i
    bm = m
print(bi.shape)
print(bm.shape)

idx = 0
plt.subplot(121)
plt.imshow(bi[idx])
plt.subplot(122)
plt.imshow(bm.numpy().reshape(1,388,388,21)[idx,...,0])
plt.show()

### 載入UNet模型

In [ ]:
unet = UNet(input_shape=(572,572,3), classes_num=21)
unet.summary()

In [ ]:
unet.compile(optimizer=tf.keras.optimizers.Adam(),
             loss="categorical_crossentropy")
unet.fit(train_dataset)